In [ ]:
import numpy as np
import string

np.random.seed(1234)

In [ ]:
initial = {} # start of a phrase
first_order = {} # second word only
second_order = {}
third_order = {}

- The first two empty strings (' ', ' ') indicate that we are not replacing any specific characters with others.
- The third argument, string.punctuation, specifies the characters we want to remove

In [ ]:
def remove_punctuation(s):
    return s.translate(str.maketrans('','',string.punctuation))

In [ ]:
def add2dict(d, k, v):
  if k not in d:
    d[k] = []
  d[k].append(v)

# [cat, cat, dog, dog, dog, dog, dog, mouse, ...]

In [ ]:
'''
for line in open('robert_frost.txt'):
  tokens = remove_punctuation(line.rstrip().lower()).split()

  T = len(tokens)
  for i in range(T):
    t = tokens[i]
    if i == 0: # predicting 1st word
      # 'initial' dictstores the count of each word appearing as the first word of a line.
      # The keys in this dictionary are the words, and the values are the counts.
      initial[t] = initial.get(t, 0.) + 1
    else:
      t_1 = tokens[i-1]
      if i == T - 1: # predicting last word
        # measure probability of ending the line
        add2dict(second_order, (t_1, t), 'END')
      if i == 1:  # predicting 2nd word given 1st word
        # measure distribution of second word
        add2dict(first_order, t_1, t)
      else: # if not predicting 1st, 2nd or last word -> apply second_order
        t_2 = tokens[i-2]
        add2dict(second_order, (t_2, t_1), t)
'''

"\nfor line in open('robert_frost.txt'):\n  tokens = remove_punctuation(line.rstrip().lower()).split()\n\n  T = len(tokens)\n  for i in range(T):\n    t = tokens[i]\n    if i == 0: # predicting 1st word\n      # 'initial' dictstores the count of each word appearing as the first word of a line.\n      # The keys in this dictionary are the words, and the values are the counts.\n      initial[t] = initial.get(t, 0.) + 1\n    else:\n      t_1 = tokens[i-1]\n      if i == T - 1: # predicting last word\n        # measure probability of ending the line\n        add2dict(second_order, (t_1, t), 'END')\n      if i == 1:  # predicting 2nd word given 1st word\n        # measure distribution of second word\n        add2dict(first_order, t_1, t)\n      else: # if not predicting 1st, 2nd or last word -> apply second_order\n        t_2 = tokens[i-2]\n        add2dict(second_order, (t_2, t_1), t)\n"

In [ ]:
for line in open("robert_frost.txt"):
  tokens = remove_punctuation(line.rstrip().lower()).split()

  T = len(tokens)
  for i in range(T):
    t = tokens[i]
    if i == 0:
      initial[t] = initial.get(t, 0.) + 1
    else:
      t_1 = tokens[i-1]
      if i == T - 1:
        add2dict(second_order, (t_1, t), 'END')
      if i == 1:
        add2dict(first_order, t_1, t)
      if i > 1:
        t_2 = tokens[i-2]
        add2dict(second_order, (t_2, t_1), t)
      if i > 2:
        t_3 = tokens[i-3]
        add2dict(third_order, (t_3, t_2, t_1), t)

In [ ]:
list(initial.items())[:10]

[('two', 8.0),
 ('and', 129.0),
 ('to', 50.0),
 ('then', 12.0),
 ('because', 1.0),
 ('though', 7.0),
 ('had', 4.0),
 ('in', 29.0),
 ('oh', 4.0),
 ('yet', 3.0)]

In [ ]:
list(first_order.items())[0]

('two',
 ['roads', 'roads', 'miles', 'oldbelievers', 'winds', 'weeks', 'of', 'at'])

In [ ]:
list(second_order.items())[0]

(('two', 'roads'), ['diverged', 'diverged'])

In [ ]:
list(third_order.items())[0]

(('two', 'roads', 'diverged'), ['in', 'in'])

In [ ]:
# normalize the distributions
initial_total = sum(initial.values())
for t, c in initial.items():
    initial[t] = c / initial_total

In [ ]:
list(initial.items())[0]

('two', 0.005571030640668524)

In [ ]:
# convert [cat, cat, cat, dog, dog, dog, dog, mouse, ...]
# into {cat: 0.5, dog: 0.4, mouse: 0.1}

def list2pdict(ts):
  # turn each list of possibilities(possible tokens) into a dictionary of probabilities
  d = {}
  n = len(ts)
  for t in ts: # count
    d[t] = d.get(t, 0.) + 1
  for t, c in d.items(): # normalize to get probs
    d[t] = c / n
  return d

In [ ]:
for t_1, ts in first_order.items():
  # replace list with dictionary of probabilities
  first_order[t_1] = list2pdict(ts)

In [ ]:
for k, ts in second_order.items():
  second_order[k] = list2pdict(ts)

for k, ts in third_order.items():
  third_order[k] = list2pdict(ts)

In [ ]:
def sample_word(d):
  '''
  simulates picking a word randomly from this bag, where words with higher probabilities have a greater chance of being selected
  '''
  # dict d has tokens as keys and their corresponding probabilities as values.
  p0 = np.random.random()
  # p0 is random number between 0 and 1
  cumulative = 0
  for t, p in d.items():
    cumulative += p # adds the probability p of the current word t to the cumulative variable
    if p0 < cumulative:
      return t
  assert(False) # should never get here

In [92]:
def generate():
  for i in range(4): # generate 4 lines
    sentence = []

    # initial word
    w0 = sample_word(initial)
    sentence.append(w0)

    # sample second word
    w1 = sample_word(first_order[w0])
    sentence.append(w1)

    if (w0, w1) in second_order:
      w2 = sample_word(second_order[(w0, w1)])
      sentence.append(w2)
    else:
      print(' '.join(sentence))
      continue

    # third-order transitions until END
    while True:
      if len(sentence) > 20:
        break
      if (w0, w1, w2) in third_order:
        w3 = sample_word(third_order[(w0, w1, w2)])
        if w3 == 'END':
          break
        sentence.append(w3)
        w0, w1, w2 = w1, w2, w3
      else:
        break
    print(' '.join(sentence))

In [93]:
generate()

brown lived at such a time
to let on he was plagued to death with me
and since it was coming up had to come
and perhaps hear some word about the weather
